# Retrieval Argumented Generation (RAG)

Retrieval
To use RAG, we need to have a database of documents that can provide relevant information for our queries. In this tutorial, we will create a database from the book "How to Build a Career in AI" by Andrew NG. We will use Langchain, Chroma, and Hugging Face to perform RAG on this book



The process of creating a database involves the following steps:

Chunking: We divide the book into smaller pieces, such as paragraphs or sentences, that can be easily indexed and retrieved.

Embedding: We use a pre-trained model from Hugging Face to convert each chunk into a vector representation, also known as a sentence embedding. This captures the semantic meaning of the chunk and allows us to compare it with other chunks or queries.

Indexing: We store the vector embeddings in a vector database, such as Chroma, that can efficiently perform similarity search. This means that given a query vector, we can find the most similar vectors in the database, and retrieve the corresponding chunks.

In [ ]:
# install the vector database, langchain, pypdf, hugging face sentence_transformers
!pip install chromadb langchain pypdf sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 112.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 19.5 MB/s e

In [ ]:
# install langchain experimental features (this will likely be moved to stable in the future).
!pip install --quiet langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.docstore.document import Document

# Load the pdf file... By default will split into pages
#loader = PyPDFLoader("How to Build a Career in AI.pdf")
loader = PyPDFLoader("ml.pdf")
pages = loader.load_and_split()

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

# Load an embedding model from hugging face.
model_name = "BAAI/bge-large-en-v1.5"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}

embed_model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker

# create a Semantic text splitter for the document, At a high level,
# this splits into sentences, then groups into groups of 3 sentences,
# and then merges ones that are similar in the embedding space.
text_splitter = SemanticChunker(embed_model)

# split the pages using Semantic Chunker.
documents = text_splitter.split_documents(pages)

In [ ]:
from langchain_community.vectorstores import Chroma

# embed and insert all chunks of the documents into the vector database
vector_db = Chroma.from_documents(
    documents,
    embed_model, # model to use for embedding the document chunks before storing.
    persist_directory='vector_db', # persist the database in memory.
    #collection_name='ai_career', # name of the collection to store the chunks in.
    collection_name='ml_interview'
)

In [ ]:
# perform a vector similarity search on a query.
query = "Define precision and recall"

# return the chunks of the most similar five embeddings in the db
docs = vector_db.similarity_search(query, k=5)

print(docs[4].page_content)

Q1: What’s the trade-off between bias and variance? Answer: Bias is error due to erroneous or overly simplistic assumptions in the learning algorithm you’re using. This can lead to the model  underfitting your data, making it hard for it to have high predictive accuracy and for 
you to generalize your knowledge from the training set to the test set. Variance is error due to too much complexity in the learning algorithm you’re using. This leads to the algorithm 
being highly sensitive to high degrees of variation in your training data, which can lead your model 
to overfit the data. You’ll b e carrying too much noise from your training data for your model to be very useful 
for your test data. The bias-variance decomposition essentially decomposes the learning error from any algorithm by adding the 
bias, the variance and a bit of irreducible er ror due to noise in the underlying dataset. Essentially, if you make 
the model more complex and add more variables, you’ll lose bias but gain 

https://www.kaggle.com/code/gpreda/rag-using-llama-2-langchain-and-chromadb
https://medium.com/@romanbessouat/enhancing-your-llm-with-rag-a-python-implementation-guide-aeab2d5be3d3
https://www.matillion.com/blog/step-by-step-guide-building-a-rag-model-with-open-source-llm-llama-2-and-vector-store-faiss
